In [1]:
# Standard modules
import os
import shutil
from importlib import reload

# Willow Fox Fortino's modules
import GPRutils
import vK2KGPR
import plotGPR

# Professor Gary Bernstein's modules
import getGaiaDR2 as gaia
import gbutil

# Science modules
import numpy as np
import astropy.units as u
import astropy.constants as c
import astropy.table as tb
import astropy.coordinates as co
import astropy.io.fits as fits
import astropy.stats as stats
from astropy.time import Time
from scipy.spatial.ckdtree import cKDTree
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from IPython import embed

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
expNum=364215
zoneDir="/data3/garyb/tno/y6/zone134"
tile0="DES2203-4623_final.fits"
earthRef="/home/fortino/y6a1.exposures.positions.fits.gz"
tileRef="/home/fortino/expnum_tile.fits.gz"
tol=0.5*u.arcsec

In [3]:
# Load in data from a reference tile (tile0). This tile is arbitrary.
# At the time of implementation, I do not have access to a reference
# file relating tiles to zones, and exposures to tiles. I only have a
# reference file (tileRef) that relates exposures to tiles. Therefore,
# this block of code opens a tile (tile0) that is actually in our
# (arbitrary) zone of interest for this thesis (zone 134). From this
# tile I can pick one of its constituent exposures. This way I have
# chosen an exposure that I know is in zone 134 (the zone I have
# access to).
file0 = os.path.join(zoneDir, tile0)
tab0 = tb.Table.read(file0)
if expNum is None:
    expNum = np.unique(tab0["EXPNUM"])[10]

In [4]:
# Use earthRef to find the center (ra, dec) of the exposure as well as
# the MJD of the exposure.
pos_tab = tb.Table.read(earthRef, hdu=1)
pos_tab = pos_tab[pos_tab["expnum"] == expNum]
ra0 = pos_tab["ra"][0]*u.deg
dec0 = pos_tab["dec"][0]*u.deg
DES_obs = Time(pos_tab["mjd_mid"][0], format="mjd")

In [5]:
# Use tileRef to find all of the tiles that our exposure is a part of.
tiles_tab = tb.Table.read(tileRef)
tiles = tiles_tab[tiles_tab["EXPNUM"] == expNum]["TILENAME"]

In [6]:
# Create an empty astropy table with all of the necessary columns.
DES_tab = tab0.copy()
DES_tab.remove_rows(np.arange(len(DES_tab)))

# Loop through each tile, open the table, and append (with tv.vstack)
# the data to our empty table. Also check if the selected exposure is
# in the Y band because we do not want to use those.
for tile in tiles:
    try:
        tile = str(tile) + "_final.fits"
        file = os.path.join(zoneDir, tile)
        tab = tb.Table.read(file)
        tab = tab[tab["EXPNUM"] == expNum]
        band = np.unique(tab["BAND"])[0]
        assert band != "y", "This exposure is in the y band. No."
        DES_tab = tb.vstack([DES_tab, tab])
    except FileNotFoundError:
        # print(f"File not found: {file}, continuing without it")
        # continue
        print(f"File not found: {file}, quitting...")
        quit()

print(f"Exposure: {expNum}")
print(f"Band: {np.unique(DES_tab['BAND'])[0]}")
print(f"Number of objects: {len(DES_tab)}")

# Initialize variables for the relevant columns.
DES_ra = np.array(DES_tab["NEW_RA"])*u.deg
DES_dec = np.array(DES_tab["NEW_DEC"])*u.deg
DES_err = np.array(DES_tab["ERRAWIN_WORLD"])*u.deg

Exposure: 364215
Band: z
Number of objects: 110646


In [7]:
# Retrieve Gaia data and initialize variables for the relevant
# columns.
GAIA_tab = gaia.getGaiaCat(ra0.value, dec0.value, 2.5, 2.5)

INFO: Query finished. [astroquery.utils.tap.core]


In [8]:
GAIA_obs = Time("J2015.5", format="jyear_str", scale="tcb")

# Adjust Gaia RA values to be between -180 and 180
GAIA_ra = np.array(GAIA_tab["ra"])*u.deg
GAIA_ra[GAIA_ra > 180*u.deg] -= 360*u.deg

GAIA_dec = np.array(GAIA_tab["dec"])*u.deg

GAIA_pmra_cosdec = np.array(GAIA_tab["pmra"])*u.mas/u.yr
GAIA_pmdec = np.array(GAIA_tab["pmdec"])*u.mas/u.yr
GAIA_parallax = np.array(GAIA_tab["parallax"])*u.mas

# Circular error approximation
GAIA_err = np.array(GAIA_tab["error"])*u.deg

# Full covariance matrix
GAIA_cov = np.array(GAIA_tab["cov"])*u.mas
GAIA_cov = np.reshape(GAIA_cov, (GAIA_cov.shape[0], 5, 5))

In [9]:
# Initialize astropy SkyCoord objects to take advantage of astropy's
# `match_coordinates_sky`.
X_DES = co.SkyCoord(DES_ra, DES_dec)
X_GAIA = co.SkyCoord(GAIA_ra, GAIA_dec)

# Match DES objects with Gaia counterparts based on how close together
# they are on the sky. 
idx, sep2d, dist3d = co.match_coordinates_sky(X_GAIA, X_DES)

# slice that can index the Gaia catalog for only the stars that have a
# match
ind_GAIA = np.where(sep2d < tol)[0]

# slice that can index the DES catalog for only the stars that have a
# match. Will be in the same order as ind_GAIA
ind_DES = idx[ind_GAIA]

print(f"There were {ind_GAIA.size} matches within {tol}.")

There were 10847 matches within 0.5 arcsec.


In [10]:
M = np.array([
[-np.sin(ra0), np.cos(ra0), 0],
[-np.cos(ra0)*np.sin(dec0), -np.sin(ra0)*np.sin(dec0), np.cos(dec0)],
[np.cos(ra0)*np.cos(dec0), np.sin(ra0)*np.cos(dec0), np.sin(dec0)]
])


X_ICRS_DES = np.array([
    np.cos(DES_dec) * np.cos(DES_ra),
    np.cos(DES_dec) * np.sin(DES_ra),
    np.sin(DES_dec)
])
X_ICRS_GAIA = np.array([
    np.cos(GAIA_dec) * np.cos(GAIA_ra),
    np.cos(GAIA_dec) * np.sin(GAIA_ra),
    np.sin(GAIA_dec)
])

xproj, yproj, zproj = np.dot(M, X_ICRS_DES)
X_gn_DES = ((xproj/zproj, yproj/zproj)*u.rad).to(u.deg).T

xproj, yproj, zproj = np.dot(M, X_ICRS_GAIA)
X_gn_GAIA = ((xproj/zproj, yproj/zproj)*u.rad).to(u.deg).T

In [11]:
# Calculate gnomonic projection of the observatory coordinates
X_E = pos_tab["observatory"][0]
X_gn_E = np.dot(M, X_E)

# Calculate time difference between DES and Gaia observations.
dt = DES_obs - GAIA_obs
dt = dt.sec
dt = (dt*u.s).to(u.yr).value

# Calculate coefficient matrix in appropriate units.
A = np.array([
    [1, 0, dt, 0, X_gn_E[0]],
    [0, 1, 0, dt, X_gn_E[1]]
])

# Calculate the variable array
X = np.vstack([
    X_gn_GAIA[:, 0].value,
    X_gn_GAIA[:, 1].value,
    GAIA_pmra_cosdec.to(u.deg/u.yr).value,
    GAIA_pmdec.to(u.deg/u.yr).value,
    GAIA_parallax.to(u.deg).value
])

# Perform epoch transformation
X_gn_transf_GAIA = np.dot(A, X).T*u.deg

In [12]:
# Find covariance matrix for X_gn_transf_GAIA.
cov = np.dot(A, np.dot(GAIA_cov, A.T))

# Swap axes to make the shape of the array more natural to humans.
cov = np.swapaxes(cov, 1, 0)

In [13]:
# Declare attribute for the full X array (DES star positions). This array will be split into the training, validation, and prediction sets.
X = X_gn_DES

# This line performs the subtraction between Gaia stars (indexed by
# ind_GAIA which takes only the Gaia stars that have DES counterparts)
# and DES stars (indexed by ind_DES which takes only the DES stars
# that have DES counterparts). This creates the astrometric residuals
# between the "true" star positions (Gaia) and the "observed" star
# positions (DES).
Y = X_gn_transf_GAIA[ind_GAIA] - X_gn_DES[ind_DES]

# This is an array of (2, 2) covariance matrices. This line selects
# only the covariance matrices for Gaia stars that have a DES
# counterpart.
E_GAIA = cov[ind_GAIA, :, :]

# Declare attribute for circular measurement error for DES stars.
# There is not a (2, 2) covariance matrix for each detection because
# DES only provides circular measurement errors (to my knowledge as a
# lowly undergraduate).
E_DES = DES_err

In [14]:
nSigma = 4
train_size = 0.80
subSample = 1.0
random_state = 0

In [15]:
rng = np.random.RandomState(random_state)

# Sigma clip on the residuals, hopefully removing a few outliers.
mask = stats.sigma_clip(Y, sigma=nSigma, axis=0).mask
mask = ~np.logical_or(*mask.T)

# Make intermediate `tv` arrays (training/validation). These arrays
# will be split into the training and validation sets and are not used
# directly. Also apply the sigma clipping mask.
# The sigma clipping mask is not applied to the attributes
# dataContainer.X, dataContainer.Y, etc., because I want to save the
# full array when I save all of this data to an npz file that way I
# can just run this method again and recreate the training,
# validation, and prediction sets (and sigma clipping won't happen
# twice). Having to run this method again when loading in npz data is
# not a bad thing because it means I don't have to save those arrays
# in the npz file.
X_tv = X[ind_DES][mask]
Y_tv = Y[mask]
E_tv_GAIA = E_GAIA[mask, :, :]
E_tv_DES = E_DES[ind_DES][mask]

# Generate an array of numbers from 0 to nTV-1.
nTV = X_tv.shape[0]
tv_mask = np.arange(nTV)

# Shuffle these numbers. Now I can take the first X% of these and have a
# random slice of the data. If I were to simply  take the first X% of the data
# arrays I would not have a random subset because the arrays are ordered.
rng.shuffle(tv_mask)

# Figure out the number of data points that should be included based on
# subSample
nSubSample = int(np.ceil(nTV*subSample))

# Take only the first nSubSample data points from the mask. Now I can take the
# first X% of these and have a random slice of (subSample)% of the full
# dataset.
tv_mask = tv_mask[:nSubSample]

# Redefine nTV based on subSample because nTrain and nValid are calculated
# from it.
nTV = int(np.ceil(nTV*subSample))
nTrain = int(np.floor(nTV*train_size))
nValid = nTV - nTrain

# Get the training set and validation set slices. When used to index the data
# arrays, these will retrieve random subsets of the data.
train_mask = tv_mask[:nTrain]
valid_mask = tv_mask[nTrain:]

# Get the training and validation sets from the slices.
Xtrain, Xvalid = X_tv[train_mask, :], X_tv[valid_mask, :]
Ytrain, Yvalid = Y_tv[train_mask, :], Y_tv[valid_mask, :]
Etrain_GAIA, Evalid_GAIA = E_tv_GAIA[train_mask,...],E_tv_GAIA[valid_mask,...]
Etrain_DES, Evalid_DES = E_tv_DES[train_mask], E_tv_DES[valid_mask]

# From the datasets (length nData) for X (astrometric position) and
# E_DES (measurement error), remove the elements that have a Gaia
# counterpart (these are already accounted for in the training
# validation sets).
Xpred = np.delete(X, ind_DES, axis=0)
Epred = np.delete(E_DES, ind_DES, axis=0)

In [54]:
Y.to(u.mas)

<Quantity [[ -1.9604484 ,   0.56078614],
           [ -8.91331912,   1.06030237],
           [  3.91558373,  13.85359899],
           ...,
           [ -5.65074296,  -9.91400461],
           [  2.23521928,   4.87410465],
           [-23.52307331, -16.13547439]] mas>

In [27]:
N = Etrain_GAIA.shape[0]
out = np.zeros((N, N, 2, 2))
out[:, :, 0, 0] = np.diag(Etrain_GAIA[:, 0, 0])
out[:, :, 1, 1] = np.diag(Etrain_GAIA[:, 1, 1])
out[:, :, 1, 0] = np.diag(Etrain_GAIA[:, 1, 0])
out[:, :, 0, 1] = np.diag(Etrain_GAIA[:, 0, 1])
W_GAIA = np.swapaxes(out, 1, 2).reshape((2*N, 2*N))

E_DES = np.vstack([Etrain_DES, Etrain_DES]).T
W_DES = np.diag(GPRutils.flat(E_DES)**2)

In [49]:
W_GAIA[:4, :4]

array([[0.07859015, 0.06450855, 0.        , 0.        ],
       [0.06450855, 0.181577  , 0.        , 0.        ],
       [0.        , 0.        , 0.02815337, 0.01862602],
       [0.        , 0.        , 0.01862602, 0.03858054]])

In [50]:
W_DES[:4, :4]

array([[2.37702103e-13, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.37702103e-13, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 9.26660588e-14, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.26660588e-14]])

In [51]:
W = W_GAIA**2 + W_DES**2

In [52]:
W[:4, :4]

array([[0.00617641, 0.00416135, 0.        , 0.        ],
       [0.00416135, 0.03297021, 0.        , 0.        ],
       [0.        , 0.        , 0.00079261, 0.00034693],
       [0.        , 0.        , 0.00034693, 0.00148846]])